# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [33]:
# import libraries
!pip3 install sqlalchemy
!pip3 install nltk
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
import string
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'stopwords'])
nltk.download('averaged_perceptron_tagger')
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

[nltk_data] Downloading package punkt to /home/aturupcu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aturupcu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aturupcu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/aturupcu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('labeled_messages', engine)
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
stop_words = nltk.corpus.stopwords.words("english")

def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    return [lemmatizer.lemmatize(word).lower().strip() for word in tokens if word not in stop_words]



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [6]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f9ed0d88430>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
from sklearn.metrics import classification_report
y_pred = pipeline.predict(X_test)

In [8]:
for i, col in enumerate(y_test):
    print(classification_report(y_test[col], y_pred[:, i],zero_division=0))

              precision    recall  f1-score   support

           0       0.68      0.51      0.59      1578
           1       0.86      0.92      0.89      4967

    accuracy                           0.82      6545
   macro avg       0.77      0.72      0.74      6545
weighted avg       0.81      0.82      0.82      6545

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5447
           1       0.84      0.48      0.61      1098

    accuracy                           0.90      6545
   macro avg       0.87      0.73      0.77      6545
weighted avg       0.89      0.90      0.88      6545

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6515
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545

              preci

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
from sklearn.model_selection import GridSearchCV


parameters = {'clf__max_depth': [10, 20, None],
              'clf__min_samples_leaf': [1, 2, 4],
              'clf__min_samples_split': [2, 5, 10],
              'clf__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)



In [10]:
cv.fit(X_train, y_train)


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f9ed0d88430>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'clf__max_depth': [10, 20, None],
                         'clf__min_samples_leaf': [1, 2, 4],
                         'clf__min_samples_split': [2, 5, 10],
                         'clf__n_estimators': [10, 20, 40]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [18]:
# predict on test data
y_pred = pipeline.predict(X_test)

labels = np.unique(y_pred)
#confusion_mat = confusion_matrix(y_test, y_pred)
confusion_mat =confusion_matrix( y_test.values.argmax(axis=1), y_pred.argmax(axis=1))
accuracy = (y_pred == y_test).mean()
print("Labels:", labels)
print("Confusion Matrix:\n", confusion_mat)
print("Accuracy:", accuracy)
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))    

Labels: [0 1]
Confusion Matrix:
 [[6545]]
Accuracy: related                   0.824905
request                   0.897021
offer                     0.995416
aid_related               0.754316
medical_help              0.920092
medical_products          0.950649
search_and_rescue         0.975401
security                  0.979679
military                  0.968067
child_alone               1.000000
water                     0.949121
food                      0.923453
shelter                   0.929565
clothing                  0.982277
money                     0.976471
missing_people            0.987471
refugees                  0.963178
death                     0.956761
other_aid                 0.872116
infrastructure_related    0.938426
transport                 0.953858
buildings                 0.951261
electricity               0.980901
tools                     0.992972
hospitals                 0.989610
shops                     0.996028
aid_centers               0.988235
oth

/home/aturupcu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aturupcu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
y_pred_test = cv.predict(X_test)
y_pred_train = cv.predict(X_train)
print(classification_report(y_test.values, y_pred_test, target_names=y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      4967
               request       0.83      0.46      0.59      1098
                 offer       0.00      0.00      0.00        30
           aid_related       0.81      0.52      0.63      2720
          medical_help       0.69      0.02      0.03       527
      medical_products       1.00      0.03      0.05       334
     search_and_rescue       1.00      0.01      0.02       162
              security       0.33      0.01      0.01       132
              military       0.56      0.02      0.04       214
           child_alone       0.00      0.00      0.00         0
                 water       0.93      0.21      0.34       432
                  food       0.86      0.42      0.57       756
               shelter       0.89      0.18      0.30       573
              clothing       1.00      0.03      0.05       119
                 money       0.50      

/home/aturupcu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aturupcu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aturupcu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aturupcu/.local/lib/python3.8/site-p

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [28]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [31]:
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_fitted = pipeline2.fit(X_train, y_train)

y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=Y.columns.values))

IndexError: list index out of range

### 9. Export your model as a pickle file

In [ ]:
m = pickle.dumps('classifier.pkl')


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.